# Creating a Heightprofile

### About this Jupyter Notebook
This Jupyter Notebook creates a heightprofile of any given location with at the measured heights (0.5m, 1m, 2m and 3m). 

### Measurment device dataset
This dataset contains temperature and relative air humidity at the heights: 0.5m, 1m, 2m and 3m.

### Aim
The goal of this Jupyter Notebook is to create a heightprofile by connecting the mesured values for relative air humidity and temperature at the given heights with a cubic-polynomial. The derivation of each parameter and reason on why the use of a cubic-polynomial is given in the main worksheet. The aim is to create a profile of each measurment-location. To enhance the figure, the final plot should include a profile of the maximum, minimum and mean value for every height over the whole measurment. These points will as well be connected with a cubic-polinomial.

----
### Import necessary packeges
In the very first step the packeges CSV and DataFrames to load-in the datasets and storeing the values inside a dataframe are imported. The packege Statistics is imported as well. This packege is used to calculate maximum, minimum and mean values from the dataset given. Finally the plot packege is needed to create the plots.

In [2]:
using CSV
using DataFrames

In [3]:
using Statistics
using Plots

## Loading the Dataset

In this step the dataset is loaded into julia. Simultaneously the values are stored inside a dataframe such that calculations inside the table can take place.

_Note:_ Bias-correction over the dataset was taken place beforehand.

In [4]:
dataset = DataFrame(CSV.File("Distanz_&_Hohe_GR_Julia.csv"))

Row,Messung,Jahr,Monat,Tag,Longitude,Latitude,Height,Stunde,Minute,Sekunde,Sensor1_T,Sensor1_LF,Sensor2_T,Sensor2_LF,Sensor3_T,Sensor3_LF,Sensor4_T,Sensor4_LF,Time,Distanz_Nachbarpunkt,Distanz_Runsum,Height_GIS1,LCZ
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Int64
1,1,2024,12,12,7.436,46.9544,573.8,10,37,14,1.76,66.78,1.91,68.44,1.3,69.57,1.24,70.79,34634,0.0,0.0,557.9,1
2,2,2024,12,12,7.43598,46.9544,573.7,10,37,16,1.8,66.87,1.86,68.44,1.3,69.56,1.27,70.81,34636,1.88488,1.88488,557.9,1
3,3,2024,12,12,7.43597,46.9544,573.6,10,37,17,1.78,66.93,1.94,68.46,1.33,69.64,1.21,70.78,34637,1.34718,3.23206,557.9,1
4,4,2024,12,12,7.43596,46.9544,573.7,10,37,18,1.76,66.94,1.88,68.4,1.29,69.51,1.21,70.73,34638,0.761114,3.99318,557.9,1
5,5,2024,12,12,7.43594,46.9544,573.6,10,37,19,1.78,66.88,1.91,68.44,1.3,69.59,1.22,70.78,34639,1.88488,5.87806,557.93,1
6,6,2024,12,12,7.43593,46.9544,573.6,10,37,20,1.75,66.81,1.94,68.49,1.26,69.5,1.19,70.78,34640,1.34718,7.22524,557.88,1
7,7,2024,12,12,7.4359,46.9544,573.4,10,37,22,1.78,66.86,1.94,68.45,1.24,69.53,1.21,70.83,34642,2.53954,9.76478,557.91,1
8,8,2024,12,12,7.43589,46.9545,573.2,10,37,23,1.76,66.82,1.89,68.48,1.24,69.54,1.18,70.77,34643,1.34718,11.112,557.93,1
9,9,2024,12,12,7.43588,46.9545,573.2,10,37,24,1.75,66.88,1.86,68.39,1.23,69.46,1.19,70.8,34644,0.761114,11.8731,557.89,1


## Creating a subset

To reduce the volume of the dataset from 3894 seperate locations to 354, in this step an new array gets created. The array contains 8 columns and 354 rows. Each column represents each measurment device (1. sensor 1 temperature, 2. sensor 1 air humidity, 3. sensor 2 temperature, ...). Each row represents the mean value of 11 consecuetive measurment locations. This step is done to smooth the variability of the variability of the curves as well as to enhance the visibility of the final figure.

In [5]:
data = Array{Float64}(undef,354,9)

for i = 1:354
    ta = dataset[11*i-10:i*11,11]
    tb = dataset[11*i-10:i*11,13]
    tc = dataset[11*i-10:i*11,15]
    td = dataset[11*i-10:i*11,17]
    la = dataset[11*i-10:i*11,12]
    lb = dataset[11*i-10:i*11,14]
    lc = dataset[11*i-10:i*11,16]
    ld = dataset[11*i-10:i*11,18]
    color = dataset[11*i-10:i*11,23]
    data[i,1] = mean(ta)
    data[i,3] = mean(tb)
    data[i,5] = mean(tc)
    data[i,7] = mean(td)
    data[i,2] = mean(la)
    data[i,4] = mean(lb)
    data[i,6] = mean(lc)
    data[i,8] = mean(ld)
    if mean(color) > 1.5
        data[i,9] = 2
    end
    if mean(color) < 1.5
        data[i,9] = 1
    end
end
print(data)

[1.7618181818181817 66.86363636363636 1.9018181818181816 68.44454545454546 1.2718181818181817 69.53545454545454 1.2072727272727273 70.78363636363635 1.0; 1.699090909090909 67.01363636363635 1.853636363636364 68.5509090909091 1.2236363636363634 69.58090909090907 1.17 70.86999999999999 1.0; 1.6818181818181819 67.37181818181817 1.830909090909091 68.86999999999999 1.2145454545454546 69.82454545454544 1.1327272727272728 71.1509090909091 1.0; 1.6881818181818182 67.64727272727272 1.831818181818182 69.2681818181818 1.2145454545454546 70.09818181818183 1.1427272727272728 71.42272727272729 1.0; 1.6690909090909087 67.86727272727272 1.8190909090909093 69.46090909090908 1.2145454545454546 70.31454545454544 1.1163636363636362 71.60363636363637 1.0; 1.6563636363636363 67.7690909090909 1.7918181818181815 69.31818181818183 1.2036363636363634 70.33545454545454 1.1090909090909091 71.61727272727272 1.0; 1.6154545454545457 67.75909090909092 1.7718181818181817 69.33 1.2063636363636363 70.37636363636365 1.11

## Creating a function for the cubic-polynomial parameters

In this step a function _cubic parameters_ is created. This function will be used to calculate the exact curve of the cubic polynomial with the given values at each height.

_Note:_ The standard form of a cubic-polynomial used here is: ax^3 + bx^2 + cx + d.

In [6]:
"""
The function cubic_parameters calculates the parameters a, b, c and d for a standard form a cubic polynomial.

Inputs:
    - lambda1: Value at height 0.5m
    - lambda2: Value at height 1m
    - lambda3: Value at height 2m
    - lambda4: Value at height 3m

Outputs:
    - Parameters a, b, c and d from the standard form of a cubic-polynomial
    - Standard form of a cubic-polynomial: ax^3 + bx^2 + cx + d
"""

function cubic_parameters(half_meter,meter,two_meters,three_meters)
    a = meter .- (8/15 .* half_meter) .- (2/3 .* two_meters) .+ (1/5 .* three_meters)
    b = (16/5 .* half_meter) .- (11/2 .* meter) .+ (3 .* two_meters) .- (7/10 .* three_meters)
    c = (17/2 .* meter) .- (88/15 .* half_meter) .- (10/3 .* two_meters) .+ (7/10 .* three_meters)
    d = (16/5 .* half_meter) .- (3 .* meter) .+ two_meters .- (1/5 .* three_meters)
    
    return a,b,c,d
end

cubic_parameters (generic function with 1 method)

## Applying the function on temperature and relative air humidity values

In this step all values from the data are applied on the newly created function.

In [7]:
#Temperature
a_T, b_T, c_T, d_T = cubic_parameters(data[:,1],data[:,3],data[:,5],data[:,7])

#Relative air humidity
a_LF, b_LF, c_LF, d_LF = cubic_parameters(data[:,2],data[:,4],data[:,6],data[:,8])

([0.5836969696969838, 0.5970303030303299, 0.6188484848484919, 0.7420606060605976, 0.709393939393955, 0.6078181818182031, 0.6033333333333246, 0.660666666666657, 0.6786060606060911, 0.7343636363636179  …  1.5980606060605975, 1.851030303030294, 1.852060606060597, 1.4521818181818276, 1.1507878787878933, 1.5541818181817906, 1.7022424242424137, 1.818666666666676, 1.9053333333333278, 1.7220000000000155], [-3.423545454545497, -3.452636363636522, -3.527181818181859, -4.2050909090908135, -4.0386363636363924, -3.51463636363642, -3.5086363636362634, -3.793545454545402, -3.8939090909091902, -4.150272727272636  …  -8.739272727272656, -9.892545454545356, -9.730090909090848, -7.789909090909099, -6.399272727272752, -8.16690909090893, -8.743909090909021, -9.241090909090893, -9.654727272727172, -8.83245454545456], [7.275666666666815, 7.208696969697151, 7.20415151515148, 8.250848484848376, 8.003787878787968, 7.306454545454649, 7.348939393939297, 7.7359696969696685, 7.900575757575879, 8.142090909090797  … 

## Extreme values

In this step the function _cubic parameters_ is applied on the maximum, minimum and mean value of each height for temperature and relative air humidity values over the whole dataset.

In [8]:
#Temperature
a_T_max, b_T_max, c_T_max, d_T_max = cubic_parameters(maximum(dataset.Sensor1_T),maximum(dataset.Sensor2_T),maximum(dataset.Sensor3_T),maximum(dataset.Sensor4_T))
a_T_min, b_T_min, c_T_min, d_T_min = cubic_parameters(minimum(dataset.Sensor1_T),minimum(dataset.Sensor2_T),minimum(dataset.Sensor3_T),minimum(dataset.Sensor4_T))
a_T_mean, b_T_mean, c_T_mean, d_T_mean = cubic_parameters(mean(dataset.Sensor1_T),mean(dataset.Sensor2_T),mean(dataset.Sensor3_T),mean(dataset.Sensor4_T))

#Relative air humidity
a_LF_max, b_LF_max, c_LF_max, d_LF_max = cubic_parameters(maximum(dataset.Sensor1_LF),maximum(dataset.Sensor2_LF),maximum(dataset.Sensor3_LF),maximum(dataset.Sensor4_LF))
a_LF_min, b_LF_min, c_LF_min, d_LF_min = cubic_parameters(minimum(dataset.Sensor1_LF),minimum(dataset.Sensor2_LF),minimum(dataset.Sensor3_LF),minimum(dataset.Sensor4_LF))
a_LF_mean, b_LF_mean, c_LF_mean, d_LF_mean = cubic_parameters(mean(dataset.Sensor1_LF),mean(dataset.Sensor2_LF),mean(dataset.Sensor3_LF),mean(dataset.Sensor4_LF))

println("Kurvenparameter der 'Temperaturmittelwertkurve':")
println("a = ",a_T_mean,"\n","b = ",b_T_mean,"\n","c = ",c_T_mean,"\n","d = ",d_T_mean,"\n")

println("Kurvenparameter der 'Luftfeuchtigkeitsmittelwertkurve':")
println("a = ",a_LF_mean,"\n","b = ",b_LF_mean,"\n","c = ",c_LF_mean,"\n","d = ",d_LF_mean)


Kurvenparameter der 'Temperaturmittelwertkurve':
a = 0.20390669405923717
b = -1.1251851566512614
c = 1.6590766135935682
d = 0.8850344119157667

Kurvenparameter der 'Luftfeuchtigkeitsmittelwertkurve':
a = 1.228680876562258
b = -6.532435798664743
c = 10.67655153227215
d = 65.94659732922432


## Plot

In the last step, the plots are created.

First, we create the plot for the temperature. At first the curves are created. In this step a variable x gets created which consists of 354 points reaching from 0.5 to 3. Then an Array where each point is stored inside gets created. In a third step a for-loop loops from 1 to the end of the dataset. Inside the loop a multiplication between the previus calculated parameters a, b, c and d with the x variable is executed. Note that each row of the arry will return one specific curve.

In the final step 3 more curves are created. One curve represents the minimum, the second the maximum and the last one the mean temperature over the whole route.  

In [9]:
x = range(0.5,3,354)
j = 1
k = 1

y_T2 = Array{Float64}(undef,354,46)
y_T1 = Array{Float64}(undef,354,308)
y_T = Array{Float64}(undef,354,354)

for i = 1:354
    if data[i,9] == 1
        y_T1[:,j] = a_T[i] .* x .* x .* x .+ b_T[i] .* x .* x .+ c_T[i] .* x .+ d_T[i]
        j = j + 1
    end
end

for i = 1:354
    if data[i,9] == 2
        y_T2[:,k] = a_T[i] .* x .* x .* x .+ b_T[i] .* x .* x .+ c_T[i] .* x .+ d_T[i]
        k = k + 1
    end
end

for i = 1:354
    y_T[:,i] = a_T[i] .* x .* x .* x .+ b_T[i] .* x .* x .+ c_T[i] .* x .+ d_T[i]
end

y_T_max = a_T_max .* x .* x .* x .+ b_T_max .* x .* x .+ c_T_max .* x .+ d_T_max
y_T_min = a_T_min .* x .* x .* x .+ b_T_min .* x .* x .+ c_T_min .* x .+ d_T_min
y_T_mean = a_T_mean .* x .* x .* x .+ b_T_mean .* x .* x .+ c_T_mean .* x .+ d_T_mean


354-element Vector{Float64}:
 1.4587647663071401
 1.4635879187080483
 1.4683293160972437
 1.4729893930636402
 1.4775685841961521
 1.482067324083693
 1.4864860473151773
 1.4908251884795194
 1.4950851821656328
 1.499266462962432
 ⋮
 1.2198958207020498
 1.2223020002234544
 1.2247763579085043
 1.2273193283461137
 1.2299313461251942
 1.23261284583466
 1.2353642620634295
 1.238186029400412
 1.241078582434523

In the following step the plot for the temperature is created.

In [10]:
plot(y_T1, x, color = :grey, linealpha = 0.5, label = false, title = "Vertikales Temperaturprofil", ylab = "Höhe [m]", xlab = "Temperatur [°C]", ylims = (0.5, 3), xlims = (0.75, 2))
plot!(y_T2, x, color = :green, linealpha = 0.5, label = false)
plot!(y_T_max, x, color = :red, linewidth = 2, label = false)
plot!(y_T_min, x, color = :red, linewidth = 2, label = false)
plot!(y_T_mean, x, color = :black, linewidth = 2, label = false)
plot!([8, 9], color = :grey, label = "Stadt")
plot!([10, 11], color = :green, label = "Wald")
plot!([9, 10], color = :red, label = "Spannweite")
plot!([9, 8], color = :black, label = "Mittelwert")
plot!(legend = :outertopright)

savefig("Vert_Temp.png")

"c:\\Users\\ruben\\Desktop\\Julia\\Vert_Temp.png"

In the following steps the steps above are repeated for the relative air humidity

In [12]:
x = range(0.5,3,354)

j = 1
k = 1

y_LF2 = Array{Float64}(undef,354,46)
y_LF1 = Array{Float64}(undef,354,308)
y_LF = Array{Float64}(undef,354,354)

for i = 1:354
    if data[i,9] == 1
        y_LF1[:,j] = a_LF[i] .* x .* x .* x .+ b_LF[i] .* x .* x .+ c_LF[i] .* x .+ d_LF[i]
        j = j + 1
    end
end

for i = 1:354
    if data[i,9] == 2
        y_LF2[:,k] = a_LF[i] .* x .* x .* x .+ b_LF[i] .* x .* x .+ c_LF[i] .* x .+ d_LF[i]
        k = k + 1
    end
end

for i = 1:354
    y_LF[:,i] = a_LF[i] .* x .* x .* x .+ b_LF[i] .* x .* x .+ c_LF[i] .* x .+ d_LF[i]
end

y_LF_max = a_LF_max .* x .* x .* x .+ b_LF_max .* x .* x .+ c_LF_max .* x .+ d_LF_max
y_LF_min = a_LF_min .* x .* x .* x .+ b_LF_min .* x .* x .+ c_LF_min .* x .+ d_LF_min
y_LF_mean = a_LF_mean .* x .* x .* x .+ b_LF_mean .* x .* x .+ c_LF_mean .* x .+ d_LF_mean 

354-element Vector{Float64}:
 69.80534925526449
 69.84099002627799
 69.87616300309463
 69.91087080441751
 69.9451160489497
 69.97890135539427
 70.01222934245432
 70.04510262883292
 70.07752383323314
 70.10949557435808
  ⋮
 72.10920341238538
 72.13884902628965
 72.16893029855987
 72.19944984789915
 72.23041029301055
 72.26181425259718
 72.29366434536206
 72.32596319000834
 72.35871340523906

In the following step the plot for the air humidity is created.

In [13]:
plot(y_LF1, x, color = :grey, linealpha = 0.5, label = false, title = "Vertikales relatives Luftfeuchtigkeitsprofil", ylab = "Höhe [m]", xlab = "Luftfeuchtigkeit [%]", ylims = (0.5, 3), xlims = (65, 85))
plot!(y_LF2, x, color = :green, linealpha = 0.5, label = false)
plot!(y_LF_max, x, color = :red, linewidth = 2, label = false)
plot!(y_LF_min, x, color = :red, linewidth = 2, label = false)
plot!(y_LF_mean, x, color = :black, linewidth = 2, label = false)
plot!([8, 9], color = :grey, label = "Stadt")
plot!([10, 11], color = :green, label = "Wald")
plot!([9, 10], color = :red, label = "Spannweite")
plot!([9, 8], color = :black, label = "Mittelwert")
plot!(legend = :outertopright)

savefig("Vert_LFK.png")

"c:\\Users\\ruben\\Desktop\\Julia\\Vert_LFK.png"